Для выполнения этого задания, давайте начнем с загрузки данных с сайта LIBSVM, создания оракулов для каждого набора данных и запуска методов градиентного спуска и Ньютона на каждом из них. Для удобства работы с данными, мы будем использовать библиотеку scikit-learn.
Для начала, загрузим данные и создадим оракулы:

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import LabelEncoder
from optimization.oracles import create_log_reg_oracle
from optimization.gradient_descent import gradient_descent
from optimization.newton import newton
import time
import matplotlib.pyplot as plt

# Загрузка данных
datasets = ['w8a', 'gisette', 'real-sim']
results = {}

for dataset_name in datasets:
    print(f"Processing {dataset_name} dataset...")
    # Загрузка данных и преобразование их в разреженные матрицы
    X, y = load_svmlight_file(f"data/{dataset_name}")
    X = csr_matrix(X)
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    
    # Создание оракула
    oracle = create_log_reg_oracle(X, y, regcoef=1/X.shape[0])
    
    # Запуск градиентного спуска
    start_time = time.time()
    gd_x, _, gd_history = gradient_descent(oracle, np.zeros(X.shape[1]), trace=True)
    gd_time = time.time() - start_time
    gd_func_values = gd_history['func']
    gd_grad_norms = gd_history['grad_norm']
    
    # Запуск метода Ньютона
    start_time = time.time()
    nt_x, _, nt_history = newton(oracle, np.zeros(X.shape[1]), trace=True)
    nt_time = time.time() - start_time
    nt_func_values = nt_history['func']
    nt_grad_norms = nt_history['grad_norm']
    
    results[dataset_name] = {
        'gd_time': gd_time,
        'gd_func_values': gd_func_values,
        'gd_grad_norms': gd_grad_norms,
        'nt_time': nt_time,
        'nt_func_values': nt_func_values,
        'nt_grad_norms': nt_grad_norms
    }

# Визуализация результатов
plt.figure(figsize=(18, 12))

for idx, dataset_name in enumerate(datasets, start=1):
    plt.subplot(2, len(datasets), idx)
    plt.semilogy(np.cumsum(results[dataset_name]['gd_time']), results[dataset_name]['gd_func_values'], label='GD')
    plt.semilogy(np.cumsum(results[dataset_name]['nt_time']), results[dataset_name]['nt_func_values'], label='Newton')
    plt.xlabel('Time (s)')
    plt.ylabel('Function Value')
    plt.title(f'{dataset_name} - Function Value')
    plt.legend()

    plt.subplot(2, len(datasets), len(datasets) + idx)
    plt.semilogy(np.cumsum(results[dataset_name]['gd_time']), results[dataset_name]['gd_grad_norms'], label='GD')
    plt.semilogy(np.cumsum(results[dataset_name]['nt_time']), results[dataset_name]['nt_grad_norms'], label='Newton')
    plt.xlabel('Time (s)')
    plt.ylabel('Gradient Norm')
    plt.title(f'{dataset_name} - Gradient Norm')
    plt.legend()

plt.tight_layout()
plt.show()
